In [1]:
import numpy as np
import os

In [2]:
from classes.model import Model
from classes.scenario import Scenario
from classes.catchment import Catchment
from classes.combine import Combine

In [3]:
dir_output = os.path.join('output', 'shiny')

# Temporary rubbish dir extender to prevent overwriting:
while os.path.exists(dir_output):
    dir_output += '1'

os.mkdir(dir_output)

In [4]:
merge_dicts = dict(
    df_units_input = {},
    df_transfer = {},
    df_lsoa_input = {},
    df_selected_units_input = {},
    df_selected_transfer = {},
    df_selected_lsoa_input = {},
    df_lsoa = {},
    df_units = {},
    df_selected_units = {},
    df_selected_lsoa = {},
    results_summary_by_admitting_unit = {},
    results_summary_by_lsoa = {}
)

In [5]:
units_to_include = [
    'GL13NN',
    'SN36BB',
    'BS105NB',
    'EX314JB',
    'PL68DH',
    'TR13LQ',
    'EX25DW',
    'BA13NG',
    'SP28BJ',
    'TA15DA',
    'TQ27AA',
    'BS234TQ',
    'BA214AT',
]

In [6]:
combine = Combine()

## Scenario 1

In [7]:
scenario_name = 'drip-and-ship'

In [8]:
# Catchment overwrites default values
catchment = Catchment({
    # "setup": setup,
    'name': scenario_name,
    "run_duration": 5,  # years
})

df_units = catchment.get_unit_services()

df_units = df_units.reset_index()

mask = df_units['postcode'].isin(units_to_include)
df_units.loc[mask, 'selected'] = 1

df_units = df_units.set_index('postcode')

In [9]:
df_units[df_units['selected'] == 1]

,stroke_team,short_code,ssnap_name,use_ivt,use_mt,use_msu,transfer_unit_postcode,lsoa_code,region,region_code,region_type,country,icb,icb_code,isdn,selected
postcode,,,,,,,,,,,,,,,,
BA13NG,Royal United Hospital Bath,BA,Royal United Hospital Bath,1,0,0,nearest,E01014428,"NHS Bath and North East Somerset, Swindon and ...",E38000231,SICBL,England,"NHS Bath and North East Somerset, Swindon and ...",E54000040,"Gloucester, BSW, BNSSG and Somerset",1
EX314JB,"North Devon District Hospital, Barnstaple",BP,North Devon District Hospital,1,0,0,nearest,E01020134,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
PL68DH,"Derriford Hospital, Plymouth",DF,Derriford Hospital,1,1,1,nearest,E01015092,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
EX25DW,Royal Devon and Exeter Hospital,EX,Royal Devon and Exeter Hospital,1,0,0,nearest,E01020013,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
GL13NN,"Gloucestershire Royal Hospital, Gloucester",GL,Gloucestershire Royal Hospital,1,0,0,nearest,E01022311,NHS Gloucestershire ICB - 11M,E38000062,SICBL,England,NHS Gloucestershire Integrated Care Board,E54000043,"Gloucester, BSW, BNSSG and Somerset",1
SN36BB,"Great Western Hospital, Swindon",GW,Great Western Hospital Swindon,1,0,0,nearest,E01015583,"NHS Bath and North East Somerset, Swindon and ...",E38000231,SICBL,England,"NHS Bath and North East Somerset, Swindon and ...",E54000040,"Gloucester, BSW, BNSSG and Somerset",1
TA15DA,"Musgrove Park Hospital, Taunton",MP,Musgrove Park Hospital,1,0,0,nearest,E01029302,NHS Somerset ICB - 11X,E38000150,SICBL,England,NHS Somerset Integrated Care Board,E54000038,"Gloucester, BSW, BNSSG and Somerset",1
BS105NB,North Bristol Hospital (Southmead),NB,North Bristol Hospitals,1,1,1,nearest,E01014631,"NHS Bristol, North Somerset and South Gloucest...",E38000222,SICBL,England,"NHS Bristol, North Somerset and South Gloucest...",E54000039,"Gloucester, BSW, BNSSG and Somerset",1
SP28BJ,Salisbury District Hospital,SS,Salisbury District Hospital,1,0,0,nearest,E01032000,"NHS Bath and North East Somerset, Swindon and ...",E38000231,SICBL,England,"NHS Bath and North East Somerset, Swindon and ...",E54000040,"Gloucester, BSW, BNSSG and Somerset",1


In [10]:
dict_useful_data = catchment.main(df_units)

scenario_simpy = Scenario(
    dict_useful_data['df_units'],
    dict_useful_data['df_transfer'],
    dict_useful_data['df_lsoa'],
)

model = Model(scenario_simpy)

model.run()

Combine input and output data:

In [11]:
df_input = dict_useful_data['df_lsoa'].copy()
df_results = model.results_summary_by_lsoa.copy()

df_input = df_input.reset_index()
df_input = df_input.set_index(df_results.index.names)

df_lsoa = combine.combine_inputs_and_results(
    df_input, df_results, how='left')

In [12]:
df_input = scenario_simpy.df_selected_lsoa.copy()
df_results = model.results_summary_by_lsoa.copy()

df_input = df_input.reset_index()
df_input = df_input.set_index(df_results.index.names)

df_selected_lsoa = combine.combine_inputs_and_results(
    df_input, df_results, how='left')

In [13]:
df_input = dict_useful_data['df_units'].copy()
df_results = model.results_summary_by_admitting_unit.copy()

df_input = df_input.reset_index()
df_input = df_input.rename(columns={'postcode': 'unit'})
df_input = df_input.set_index(df_results.index.names)

df_units = combine.combine_inputs_and_results(
    df_input, df_results, how='left')

In [14]:
df_input = scenario_simpy.df_selected_units.copy()
df_results = model.results_summary_by_admitting_unit.copy()

df_input = df_input.reset_index()
df_input = df_input.rename(columns={'postcode': 'unit'})
df_input = df_input.set_index(df_results.index.names)

df_selected_units = combine.combine_inputs_and_results(
    df_input, df_results, how='left')

Gather data for later combination:

In [15]:
# Update big dictionary of Dataframes:
merge_dicts['df_units_input'][scenario_name] = dict_useful_data['df_units']
merge_dicts['df_transfer'][scenario_name] = dict_useful_data['df_transfer']
merge_dicts['df_lsoa_input'][scenario_name] = dict_useful_data['df_lsoa']
merge_dicts['df_selected_units_input'][scenario_name] = scenario_simpy.df_selected_units
merge_dicts['df_selected_transfer'][scenario_name] = scenario_simpy.df_selected_transfer
merge_dicts['df_selected_lsoa_input'][scenario_name] = scenario_simpy.df_selected_lsoa
merge_dicts['results_summary_by_admitting_unit'][scenario_name] = model.results_summary_by_admitting_unit
merge_dicts['results_summary_by_lsoa'][scenario_name] = model.results_summary_by_lsoa
merge_dicts['df_lsoa'][scenario_name] = df_lsoa
merge_dicts['df_units'][scenario_name] = df_units
merge_dicts['df_selected_lsoa'][scenario_name] = df_selected_lsoa
merge_dicts['df_selected_units'][scenario_name] = df_selected_units

Save data to file:

In [16]:
dir_output_scenario = os.path.join(dir_output, scenario_name)

if os.path.exists(dir_output_scenario):
    pass
else:
    os.mkdir(dir_output_scenario)

In [17]:
file_dict = {
    'df_units_input': {
        'file': 'df_units_input.csv',
        'data': dict_useful_data['df_units']
    },
    'df_transfer': {
        'file': 'df_transfer.csv',
        'data': dict_useful_data['df_transfer']
    },
    'df_lsoa_input': {
        'file': 'df_lsoa_input.csv',
        'data': dict_useful_data['df_lsoa']
    },
    'df_selected_units_input': {
        'file': 'df_selected_units_input.csv',
        'data': scenario_simpy.df_selected_units
    },
    'df_selected_transfer': {
        'file': 'df_selected_transfer.csv',
        'data': scenario_simpy.df_selected_transfer
    },
    'df_selected_lsoa_input': {
        'file': 'df_selected_lsoa_input.csv',
        'data': scenario_simpy.df_selected_lsoa
    },
    'results_summary_by_admitting_unit': {
        'file': 'results_summary_by_admitting_unit.csv',
        'data': model.results_summary_by_admitting_unit
    },
    'results_summary_by_lsoa': {
        'file': 'results_summary_by_lsoa.csv',
        'data': model.results_summary_by_lsoa
    },
    'df_units': {
        'file': 'df_units.csv',
        'data': merge_dicts['df_units'][scenario_name]
    },
    'df_lsoa': {
        'file': 'df_lsoa.csv',
        'data': merge_dicts['df_lsoa'][scenario_name]
    },
    'df_selected_units': {
        'file': 'df_selected_units.csv',
        'data': merge_dicts['df_selected_units'][scenario_name]
    },
    'df_selected_lsoa': {
        'file': 'df_selected_lsoa.csv',
        'data': merge_dicts['df_selected_lsoa'][scenario_name]
    },
}

In [18]:
for data_type, data_dict in file_dict.items():
    path_to_file = os.path.join(dir_output_scenario, data_dict['file'])
    data_dict['data'].to_csv(path_to_file)

## Scenario 2

In [19]:
scenario_name = 'mothership'

In [20]:
# Catchment overwrites default values
catchment = Catchment({
    # "setup": setup,
    'name': scenario_name,
    "run_duration": 5,  # years
})

df_units = catchment.get_unit_services()

df_units = df_units.reset_index()

# Set all non-MT units to have no IVT:
mask = df_units['use_mt'] == 0
df_units.loc[mask, 'use_ivt'] = 0

mask = (
    (df_units['postcode'].isin(units_to_include)) &
    (df_units['use_ivt'] == 1)
)
df_units.loc[mask, 'selected'] = 1


df_units = df_units.set_index('postcode')

In [21]:
df_units[df_units['selected'] == 1]

,stroke_team,short_code,ssnap_name,use_ivt,use_mt,use_msu,transfer_unit_postcode,lsoa_code,region,region_code,region_type,country,icb,icb_code,isdn,selected
postcode,,,,,,,,,,,,,,,,
PL68DH,"Derriford Hospital, Plymouth",DF,Derriford Hospital,1,1,1,nearest,E01015092,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
BS105NB,North Bristol Hospital (Southmead),NB,North Bristol Hospitals,1,1,1,nearest,E01014631,"NHS Bristol, North Somerset and South Gloucest...",E38000222,SICBL,England,"NHS Bristol, North Somerset and South Gloucest...",E54000039,"Gloucester, BSW, BNSSG and Somerset",1


In [22]:
dict_useful_data['df_transfer']

,transfer_unit_travel_time,transfer_unit_postcode,selected
postcode,,,
SY231ER,135.8,CF144XW,0
CB20QQ,0.0,CB20QQ,0
L97AL,0.0,L97AL,0
CH495PE,27.3,L97AL,0
BA13NG,33.7,BS105NB,1
...,...,...,...
SL24HL,NaN,<NA>,0
HP112TT,34.8,OX39DU,0
BA214AT,74.5,BS105NB,1


In [23]:
dict_useful_data = catchment.main(df_units)

scenario_simpy = Scenario(
    dict_useful_data['df_units'],
    dict_useful_data['df_transfer'],
    dict_useful_data['df_lsoa'],
)

model = Model(scenario_simpy)

model.run()

Combine input and output data:

In [24]:
df_input = dict_useful_data['df_lsoa'].copy()
df_results = model.results_summary_by_lsoa.copy()

df_input = df_input.reset_index()
df_input = df_input.set_index(df_results.index.names)

df_lsoa = combine.combine_inputs_and_results(
    df_input, df_results, how='left')

In [25]:
df_input = scenario_simpy.df_selected_lsoa.copy()
df_results = model.results_summary_by_lsoa.copy()

df_input = df_input.reset_index()
df_input = df_input.set_index(df_results.index.names)

df_selected_lsoa = combine.combine_inputs_and_results(
    df_input, df_results, how='left')

In [26]:
df_input = dict_useful_data['df_units'].copy()
df_results = model.results_summary_by_admitting_unit.copy()

df_input = df_input.reset_index()
df_input = df_input.rename(columns={'postcode': 'unit'})
df_input = df_input.set_index(df_results.index.names)

df_units = combine.combine_inputs_and_results(
    df_input, df_results, how='left')

In [27]:
df_units

property                             stroke_team short_code  \
subtype                                                       
unit                                                          
SY231ER         Bronglais Hospital (Aberystwyth)         AB   
CB20QQ         Addenbrooke's Hospital, Cambridge         AD   
L97AL     University Hospital Aintree, Liverpool         AI   
CH495PE             Arrowe Park Hospital, Wirral         AR   
BA13NG                Royal United Hospital Bath         BA   
...                                          ...        ...   
SL24HL              Wexham Park Hospital, Slough         WX   
HP112TT   Wycombe General Hospital, High Wycombe         WY   
BA214AT                 Yeovil District Hospital         YE   
YO318HE                            York Hospital         YK   
LL572PW                  Ysbyty Gwynedd (Bangor)         YS   

property                   ssnap_name use_ivt use_mt use_msu  \
subtype                                                        
unit                                                           
SY231ER            Bronglais Hospital       0      0       0   
CB20QQ         Addenbrooke's Hospital       1      1       1   
L97AL     University Hospital Aintree       1      1       1   
CH495PE   Wirral Arrowe Park Hospital       0      0       0   
BA13NG     Royal United Hospital Bath       0      0       0   
...                               ...     ...    ...     ...   
SL24HL           Wexham Park Hospital       0      0       0   
HP112TT      Wycombe General Hospital       0      0       0   
BA214AT      Yeovil District Hospital       0      0       0   
YO318HE                 York Hospital       0      0       0   
LL572PW                Ysbyty Gwynedd       0      0       0   

property transfer_unit_postcode  lsoa_code  \
subtype                                      
unit                                         
SY231ER                 nearest  W01000512   
CB20QQ                  nearest  E01017995   
L97AL                   nearest  E01006654   
CH495PE                 nearest  E01007251   
BA13NG                  nearest  E01014428   
...                         ...        ...   
SL24HL                  nearest  E01016519   
HP112TT                 nearest  E01017837   
BA214AT                 nearest  E01029231   
YO318HE                 nearest  E01013343   
LL572PW                 nearest  W01000102   

property                                             region region_code  ...  \
subtype                                                                  ...   
unit                                                                     ...   
SY231ER                   Hywel Dda University Health Board   W11000025  ...   
CB20QQ        NHS Cambridgeshire and Peterborough ICB - 06H   E38000260  ...   
L97AL                 NHS Cheshire and Merseyside ICB - 99A   E38000101  ...   
CH495PE               NHS Cheshire and Merseyside ICB - 12F   E38000208  ...   
BA13NG    NHS Bath and North East Somerset, Swindon and ...   E38000231  ...   
...                                                     ...         ...  ...   
SL24HL                              NHS Frimley ICB - D4U1Y   E38000252  ...   
HP112TT   NHS Buckinghamshire, Oxfordshire and Berkshire...   E38000223  ...   
BA214AT                              NHS Somerset ICB - 11X   E38000150  ...   
YO318HE            NHS Humber and North Yorkshire ICB - 03Q   E38000188  ...   
LL572PW             Betsi Cadwaladr University Health Board   W11000023  ...   

property time_transfer_unit_arrival     time_puncture     mRS shift      \
subtype                        mean std          mean std      mean std   
unit                                                                      
SY231ER                         NaN NaN           NaN NaN       NaN NaN   
CB20QQ                          NaN NaN           NaN NaN       NaN NaN   
L97AL                           NaN NaN           NaN NaN       NaN NaN   
CH49

In [28]:
df_input = scenario_simpy.df_selected_units.copy()
df_results = model.results_summary_by_admitting_unit.copy()

df_input = df_input.reset_index()
df_input = df_input.rename(columns={'postcode': 'unit'})
df_input = df_input.set_index(df_results.index.names)

df_selected_units = combine.combine_inputs_and_results(
    df_input, df_results, how='left')

Gather data for later combination:

In [29]:
# Update big dictionary of Dataframes:
merge_dicts['df_units_input'][scenario_name] = dict_useful_data['df_units']
merge_dicts['df_transfer'][scenario_name] = dict_useful_data['df_transfer']
merge_dicts['df_lsoa_input'][scenario_name] = dict_useful_data['df_lsoa']
merge_dicts['df_selected_units_input'][scenario_name] = scenario_simpy.df_selected_units
merge_dicts['df_selected_transfer'][scenario_name] = scenario_simpy.df_selected_transfer
merge_dicts['df_selected_lsoa_input'][scenario_name] = scenario_simpy.df_selected_lsoa
merge_dicts['results_summary_by_admitting_unit'][scenario_name] = model.results_summary_by_admitting_unit
merge_dicts['results_summary_by_lsoa'][scenario_name] = model.results_summary_by_lsoa
merge_dicts['df_lsoa'][scenario_name] = df_lsoa
merge_dicts['df_units'][scenario_name] = df_units
merge_dicts['df_selected_lsoa'][scenario_name] = df_selected_lsoa
merge_dicts['df_selected_units'][scenario_name] = df_selected_units

Save data to file:

In [30]:
dir_output_scenario = os.path.join(dir_output, scenario_name)

if os.path.exists(dir_output_scenario):
    pass
else:
    os.mkdir(dir_output_scenario)

In [31]:
file_dict = {
    'df_units_input': {
        'file': 'df_units_input.csv',
        'data': dict_useful_data['df_units']
    },
    'df_transfer': {
        'file': 'df_transfer.csv',
        'data': dict_useful_data['df_transfer']
    },
    'df_lsoa_input': {
        'file': 'df_lsoa_input.csv',
        'data': dict_useful_data['df_lsoa']
    },
    'df_selected_units_input': {
        'file': 'df_selected_units_input.csv',
        'data': scenario_simpy.df_selected_units
    },
    'df_selected_transfer': {
        'file': 'df_selected_transfer.csv',
        'data': scenario_simpy.df_selected_transfer
    },
    'df_selected_lsoa_input': {
        'file': 'df_selected_lsoa_input.csv',
        'data': scenario_simpy.df_selected_lsoa
    },
    'results_summary_by_admitting_unit': {
        'file': 'results_summary_by_admitting_unit.csv',
        'data': model.results_summary_by_admitting_unit
    },
    'results_summary_by_lsoa': {
        'file': 'results_summary_by_lsoa.csv',
        'data': model.results_summary_by_lsoa
    },
    'df_units': {
        'file': 'df_units.csv',
        'data': merge_dicts['df_units'][scenario_name]
    },
    'df_lsoa': {
        'file': 'df_lsoa.csv',
        'data': merge_dicts['df_lsoa'][scenario_name]
    },
    'df_selected_units': {
        'file': 'df_selected_units.csv',
        'data': merge_dicts['df_selected_units'][scenario_name]
    },
    'df_selected_lsoa': {
        'file': 'df_selected_lsoa.csv',
        'data': merge_dicts['df_selected_lsoa'][scenario_name]
    },
}

In [32]:
for data_type, data_dict in file_dict.items():
    path_to_file = os.path.join(dir_output_scenario, data_dict['file'])
    data_dict['data'].to_csv(path_to_file)

## Combine multiple scenarios

In [35]:
combo_df_units = combine.combine_units(
    merge_dicts['df_units'])

combo_df_transfer = combine.combine_transfer(
    merge_dicts['df_transfer'])

combo_df_lsoa = combine.combine_lsoa(
    merge_dicts['df_lsoa'])

combo_df_selected_units = combine.combine_units(
    merge_dicts['df_selected_units'])

combo_df_selected_transfer = combine.combine_transfer(
    merge_dicts['df_selected_transfer'])

combo_df_selected_lsoa = combine.combine_lsoa(
    merge_dicts['df_selected_lsoa'])

/home/anna/stroke-modelling/simpy-pathway-model/classes/combine.py:404: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_any = df.copy().drop(
/home/anna/stroke-modelling/simpy-pathway-model/classes/combine.py:404: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_any = df.copy().drop(


Save data to file:

In [36]:
dir_output_scenario = os.path.join(dir_output, 'combined')

if os.path.exists(dir_output_scenario):
    pass
else:
    os.mkdir(dir_output_scenario)

In [37]:
file_dict = {
    'combo_df_units': {
        'file': 'df_units.csv',
        'data': combo_df_units,
    },
    'combo_df_transfer': {
        'file': 'df_transfer.csv',
        'data': combo_df_transfer,
    },
    'combo_df_lsoa': {
        'file': 'df_lsoa.csv',
        'data': combo_df_lsoa,
    },
    'combo_df_selected_units': {
        'file': 'df_selected_units.csv',
        'data': combo_df_selected_units, 
    },
    'combo_df_selected_transfer': {
        'file': 'df_selected_transfer.csv',
        'data': combo_df_selected_transfer,
    },
    'combo_df_selected_lsoa': {
        'file': 'df_selected_lsoa.csv',
        'data': combo_df_selected_lsoa,
    }
}

In [38]:
for data_type, data_dict in file_dict.items():
    path_to_file = os.path.join(dir_output_scenario, data_dict['file'])
    data_dict['data'].to_csv(path_to_file)